# Pawpularity with EfficientNet: [Inference]

<font size="5" color="red">If you found my work useful, please upvote!</font>

This is Inference Notebook, training notebook can be found [here]( https://www.kaggle.com/lonnieqin/pawpularity-with-efficientnet-training).

According to the training notebook, validation RMSE for different models are following:

|  Model   | Validation RMSE  |
|  ----  | ----  |
| petfinder_efficientnet_model_0.h5  | 18.96 |
| petfinder_efficientnet_model_1.h5  | 18.84 |
| petfinder_efficientnet_model_2.h5  | 18.41 |
| petfinder_efficientnet_model_3.h5  | 18.34 |
| petfinder_efficientnet_model_4.h5  | 19.04 |




I have been trying different ways of ensambe method. Here are results of different experiments, currently using mean ensamble removing best and worst models have slightly better Test RMSE score.

|  Method   | Test RMSE  |
|  ----  | ----  |
| mean ensamble of all models  | 18.75475 |
| mean ensamble of top 2 models  | 18.77554 |
| mean ensamble of top 3 models  | 18.77399 |
| mean ensamble removing best and worst models  | 18.74878 |
| mean ensamble removing worst model  | 18.75553 |
| mean ensamble removing best model  | 18.74980 |





In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
tabular_columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
image_size = 224
batch_size = 128

In [ ]:
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
test["file_path"] = test["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/test/" + identifier + ".jpg")
test.head()

### The Efficient Net Model

In [ ]:
efficient_net = tf.keras.applications.EfficientNetB0(
    weights = "../input/efficientnet-b0-for-keras-no-top/efficientnetb0_notop.h5", 
    include_top = False, 
    input_shape = (image_size, image_size, 3)
)    
efficient_net.trainable = False
efficient_net.summary()

In [ ]:
keras.utils.plot_model(efficient_net, show_shapes=True)

### The Image Prediciton Model

In [ ]:
def get_image_prediction_model(inputs):
    x = efficient_net(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.0)(x)
    return x

### The Tabular Prediciton Model

In [ ]:
def get_tabular_prediciton_model(inputs):
    width = 32
    depth = 3
    activation = "relu"
    kernel_regularizer = keras.regularizers.l2()
    x = keras.layers.Dense(
            width, 
            activation=activation,
            kernel_regularizer=kernel_regularizer
        )(inputs)
    for i in range(depth):
        if i == 0:
            x = inputs
        x = keras.layers.Dense(
            width, 
            activation=activation,
            kernel_regularizer=kernel_regularizer
        )(x)
        if (i + 1) % 3 == 0:
            x = keras.layers.Concatenate()([x, inputs])
    return x

### The Pawpularity Model

In [ ]:
def get_model():
    image_inputs = tf.keras.Input((image_size, image_size , 3))
    tabular_inputs = tf.keras.Input(len(tabular_columns))
    image_x = get_image_prediction_model(image_inputs)
    tabular_x = get_tabular_prediciton_model(tabular_inputs)
    x = tf.keras.layers.Concatenate(axis=1)([image_x, tabular_x])
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=[image_inputs, tabular_inputs], outputs=[output])
    optimizer = tf.keras.optimizers.Adam(1e-3)
    model.compile(loss=rmse, optimizer=optimizer, metrics=["mae", "mape"])
    return model

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true -  y_pred) ** 2))

In [ ]:
def preprocess_test_data(image_url, tabular):
    print(image_url, tabular)
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    # 0 won't be used in prediction, but it's needed in this senario or the tabular variable is treated as label.
    return (image, tabular), 0

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test["file_path"], test[tabular_columns])).map(preprocess_test_data).batch(batch_size).cache().prefetch(2)

In [ ]:
total_results = []
for i in [0, 1, 2, 4]:
    model = get_model()
    model.load_weights("../input/petfinder-efficientnet-model/petfinder_efficientnet_model_%d.h5"%(i))
    results = model.predict(test_ds).reshape(-1)
    total_results.append(results)
results = np.mean(total_results, axis=0).reshape(-1)
sample_submission["Pawpularity"] = results
sample_submission.to_csv("submission.csv", index=False)
